# 梯度检验

欢迎来到本周的最终作业！在此作业中，您将学习如何实施和使用梯度检验。假设您是致力于在全球范围内提供移动支付的团队的一员，并被要求构建一个深度学习模型来检测欺诈--每当有人付款时，您想要查看付款是否可能是欺诈性的，例如用户的帐户已被黑客接管。但反向传播实施起来非常具有挑战性，有时还会出现错误。由于这是一项关键任务应用程序，因此贵公司的CEO希望确保您的反向传播实施是正确的。你的CEO说，“给我一个证据证明你的反向传播确实在起作用！”为了保证这一点，您将使用“梯度检查”。我们开始做吧！


In [29]:
# 导入相关库
import numpy as np
from testCases import *
import gc_utils
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

## 一、怎么样进行梯度检验 ？

反向传播计算梯度$\frac{\partial J}{\partial \theta}$, 其中 $\theta$ 表示模型中的参数，使用前向传播和损失函数计算 $J$ 。

因为向前传播相对容易实现，所以您确信自己得到了正确的结果，从而您几乎100％确保您正确计算了价值函数$J$。 因此，您可以使用您用来计算 $J$ 的代码来检验计算$\frac{\partial J}{\partial \theta}$的代码是否正确。

让我们回头看一下导数（或梯度）的定义：
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

- $\frac{\partial J}{\partial \theta}$ 是你想确保你的计算正确的值。
- 你可以计算 $J(\theta + \varepsilon)$ 和  $J(\theta - \varepsilon)$ (在 $\theta$是一个实数的情况下), 因为你确信你对$J$的实现是正确的。

让我们用公式 (1) 和一个非常小的值$\varepsilon$ 来向你的 CEO证明你计算 $\frac{\partial J}{\partial \theta}$的代码是正确的。


## 二、 1维梯度检验

对于1维线性函数模型 $J(\theta) = \theta x$，它只包含一个实参 $\theta$,  $x$是输入。

接下来会用代码来实现$J(.)$ 和它的导数 $\frac{\partial J}{\partial \theta}$的计算。然后用梯度检验来确保对$J$的梯度计算是正确的。 

<img src="images/1Dgrad_kiank.png" style="width:600px;height:250px;">
<caption><center> <u> **图 1** </u>: **1维线性模型**<br> </center></caption>

上图显示了关键计算步骤：首先，从$x$开始计算出函数$J(x)$（前向传播）。然后，计算它的梯度$\frac{\partial J}{\partial \theta}$ （后向传播）。

**内容**: 对这个简单的函数实现前向传播和后向传播。比如，用两个独立的函数同时计算 $J(.)$和它对$\theta$的导数。

In [30]:
def forward_propagation(x,theta):
    """

    实现图中呈现的线性前向传播（计算J）（J（theta）= theta * x）

    参数：
    x  - 一个实值输入
    theta  - 参数，也是一个实数

    返回：
    J  - 函数J的值，用公式J（theta）= theta * x计算
    """
    J = np.dot(theta,x)

    return J

测试一下：

In [31]:
#测试forward_propagation
print("-----------------测试forward_propagation-----------------")
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

-----------------测试forward_propagation-----------------
J = 8


前向传播有了，我们来看一下反向传播：

In [32]:
def backward_propagation(x,theta):
    """
    计算J相对于θ的导数。

    参数：
        x  - 一个实值输入
        theta  - 参数，也是一个实数

    返回：
        dtheta  - 相对于θ的成本梯度
    """
    dtheta = x

    return dtheta

测试一下：

In [33]:
#测试backward_propagation
print("-----------------测试backward_propagation-----------------")
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

-----------------测试backward_propagation-----------------
dtheta = 2


**内容**: 为了证明 `backward_propagation()` 函数正确计算了梯度$\frac{\partial J}{\partial \theta}$, 下面实现梯度检验。

**介绍**:
- 首先，用公式（1）和一个非常小的值 $\varepsilon$来计算 "gradapprox" 。有如下2个步骤：
    1. $\theta^{+} = \theta + \varepsilon$
    2. $\theta^{-} = \theta - \varepsilon$
    3. $J^{+} = J(\theta^{+})$
    4. $J^{-} = J(\theta^{-})$
    5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$
- 然后，通过反向传播计算出梯度，并把其结果保存在变量"grad"中。
- 最后，用下面的公式计算 "gradapprox" 和 "grad" 的差值:
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$
计算这个公式需要3个步骤：
   - 1'. 利用np.linalg.norm(...)计算出分子；
   - 2'. 计算分母。需要要调用np.linalg.norm(...) 两次。
   - 3'. 让它们相除。
- 如果差值非常小 (通常小于$10^{-7}$)，就证明梯度计算正确。否则，梯度计算可能存在错误。


In [34]:
def gradient_check(x,theta,epsilon=1e-7):
    """

    实现图中的反向传播。

    参数：
        x  - 一个实值输入
        theta  - 参数，也是一个实数
        epsilon  - 使用公式（3）计算输入的微小偏移以计算近似梯度

    返回：
        近似梯度和后向传播梯度之间的差异
    """

    #使用公式（1）的左侧计算gradapprox。
    thetaplus = theta + epsilon                               # Step 1
    thetaminus = theta - epsilon                              # Step 2
    J_plus = forward_propagation(x, thetaplus)                # Step 3
    J_minus = forward_propagation(x, thetaminus)              # Step 4
    gradapprox = (J_plus - J_minus) / (2 * epsilon)           # Step 5


    #检查gradapprox是否足够接近backward_propagation（）的输出
    grad = backward_propagation(x, theta)

    numerator = np.linalg.norm(grad - gradapprox)                      # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)    # Step 2'
    difference = numerator / denominator                               # Step 3'

    if difference < 1e-7:
        print("梯度检验：梯度正常!")
    else:
        print("梯度检验：梯度超出阈值!")

    return difference

测试一下：

In [35]:
#测试gradient_check
print("-----------------测试gradient_check-----------------")
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

-----------------测试gradient_check-----------------
梯度检验：梯度正常!
difference = 2.919335883291695e-10


# 三、N维梯度检验

下图描述了你的欺诈检测模型的前向传播和后向传播。

<img src="images/NDgrad_kiank.png" style="width:600px;height:400px;">
<caption><center> <u> **图 2** </u>: **深度神经网络**<br>*LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID*</center></caption>

下面是前向传播和后向传播的实现。

In [36]:
def forward_propagation_n(X,Y,parameters):
    """
    实现图中的前向传播（并计算成本）。

    参数：
        X - 训练集为m个例子
        Y -  m个示例的标签
        parameters - 包含参数“W1”，“b1”，“W2”，“b2”，“W3”，“b3”的python字典：
            W1  - 权重矩阵，维度为（5,4）
            b1  - 偏向量，维度为（5,1）
            W2  - 权重矩阵，维度为（3,5）
            b2  - 偏向量，维度为（3,1）
            W3  - 权重矩阵，维度为（1,3）
            b3  - 偏向量，维度为（1,1）

    返回：
        cost - 成本函数（logistic）
    """
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1,X) + b1
    A1 = gc_utils.relu(Z1)

    Z2 = np.dot(W2,A1) + b2
    A2 = gc_utils.relu(Z2)

    Z3 = np.dot(W3,A2) + b3
    A3 = gc_utils.sigmoid(Z3)

    #计算成本
    logprobs = np.multiply(-np.log(A3), Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = (1 / m) * np.sum(logprobs)

    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)

    return cost, cache

def backward_propagation_n(X,Y,cache):
    """
    实现图中所示的反向传播。

    参数：
        X - 输入数据点（输入节点数量，1）
        Y - 标签
        cache - 来自forward_propagation_n（）的cache输出

    返回：
        gradients - 一个字典，其中包含与每个参数、激活和激活前变量相关的成本梯度。
    """
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache

    dZ3 = A3 - Y
    dW3 = (1. / m) * np.dot(dZ3,A2.T)
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)

    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    #dW2 = 1. / m * np.dot(dZ2, A1.T) * 2  # Should not multiply by 2
    dW2 = 1. / m * np.dot(dZ2, A1.T)
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    #db1 = 4. / m * np.sum(dZ1, axis=1, keepdims=True) # Should not multiply by 4
    db1 = 1. / m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}

    return gradients

你获得了欺诈测试集的结果，但是你对你的模型不是100%确定是正确的。没有人完美的。下面实现梯度检验来检查你的梯度是否正确。

**怎么实现梯度检验?**.

因为在内容一、二中，是通过比较“gradapprox”与反向传播计算的梯度。 该公式仍然是： 
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$
然而，$\theta$ 不再是标量。 这是一个名为“parameters”的字典。 我们为你实现了一个函数“dictionary_to_vector（）”。 它将“parameters”字典转换为一个名为“values”的向量，方法是通过将该字典的所有参数（W1，b1，W2，b2，W3，b3）整形为向量并将它们连接起来而获得。

“vector_to_dictionary”是反函数，它返回“parameters”字典。 

<img src="images/dictionary_to_vector.png" style="width:600px;height:400px;">
<caption><center> <u> **图 2** </u>: **dictionary_to_vector() and vector_to_dictionary()**<br> 在 gradient_check_n()中会需要这些函数</center></caption>

我们也用函数gradients_to_vector()把字典 "gradients" 转化为向量 "grad" 。对此不用担心。

**内容**: Implement gradient_check_n().

**介绍**: 下面是实现梯度检验的伪代码。

For each i in num_parameters:
- 计算 `J_plus[i]`:
    1. Set $\theta^{+}$ to `np.copy(parameters_values)`
    2. Set $\theta^{+}_i$ to $\theta^{+}_i + \varepsilon$
    3. Calculate $J^{+}_i$ using to `forward_propagation_n(x, y, vector_to_dictionary(`$\theta^{+}$ `))`.     
- 计算`J_minus[i]`: 用 $\theta^{-}$做同样事情 
- 计算 $gradapprox[i] = \frac{J^{+}_i - J^{-}_i}{2 \varepsilon}$

因此，得到一个向量gradapprox, 其中gradapprox[i] 是相对于 `parameter_values[i]`的梯度的近似。然后，比较向量gradapprox和反向传播的梯度向量。就像1维梯度检验 (Steps 1', 2', 3')中一样，计算：
$$ difference = \frac {\| grad - gradapprox \|_2}{\| grad \|_2 + \| gradapprox \|_2 } \tag{3}$$

In [44]:
def gradient_check_n(parameters,gradients,X,Y,epsilon=1e-7):
    """
    检查backward_propagation_n是否正确计算forward_propagation_n输出的成本梯度

    参数：
        parameters - 包含参数“W1”，“b1”，“W2”，“b2”，“W3”，“b3”的python字典：
        grad_output_propagation_n的输出包含与参数相关的成本梯度。
        x  - 输入数据点，维度为（输入节点数量，1）
        y  - 标签
        epsilon  - 计算输入的微小偏移以计算近似梯度

    返回：
        difference - 近似梯度和后向传播梯度之间的差异
    """
    #初始化参数
    parameters_values , keys = gc_utils.dictionary_to_vector(parameters) #keys用不到
    grad = gc_utils.gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters,1))
    J_minus = np.zeros((num_parameters,1))
    gradapprox = np.zeros((num_parameters,1))

    #计算gradapprox
    for i in range(num_parameters):
        #计算J_plus [i]。输入：“parameters_values，epsilon”。输出=“J_plus [i]”
        thetaplus = np.copy(parameters_values)                                                  # Step 1
        thetaplus[i][0] = thetaplus[i][0] + epsilon                                             # Step 2
        J_plus[i], cache = forward_propagation_n(X,Y,gc_utils.vector_to_dictionary(thetaplus))  # Step 3 ，cache用不到

        #计算J_minus [i]。输入：“parameters_values，epsilon”。输出=“J_minus [i]”。
        thetaminus = np.copy(parameters_values)                                                 # Step 1
        thetaminus[i][0] = thetaminus[i][0] - epsilon                                           # Step 2        
        J_minus[i], cache = forward_propagation_n(X,Y,gc_utils.vector_to_dictionary(thetaminus))# Step 3 ，cache用不到

        #计算gradapprox[i]
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)

    #通过计算差异比较gradapprox和后向传播梯度。
    numerator = np.linalg.norm(grad - gradapprox)                                     # Step 1'
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                   # Step 2'
    difference = numerator / denominator                                              # Step 3'

    
    if difference > 1e-7:
        print ("\033[93m" + "梯度计算错误！There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "梯度计算正确！Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")

    return difference

In [45]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)
print("difference = "+ str(difference))

梯度计算错误！There is a mistake in the backward propagation! difference = 1.1890417878779317e-07
difference = 1.1890417878779317e-07


我们给你的`back_propagation_n`代码似乎有错误！很好，你已经实现了梯度检验。返回`backward_propagation`并尝试查找/更正错误（提示：检查dW2和db1）。当您认为已修复时，请重新运行梯度检验。请记住，如果修改代码，则需要重新执行定义`backward_propagation_n（）`的单元格。你能通过梯度检验来证明你的导数计算是否正确？即使这部分作业没有评分，我们强烈建议您尝试找到错误并重新运行梯度检验，直到您确信后向传播已正确实现。 
    
    **注意** 
    
   - 梯度检验很慢！使用$\frac{\partial J}{\partial \theta} \approx  \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon}$ 计算近似梯度成本高昂。因此，我们不会在训练期间的每次迭代中运行梯度检验。只需检查几次梯度是否正确。 
   - 至少在我们提出的情况下，梯度检验不适用于dropout。您通常会在没有dropout的情况下运行梯度检验算法，以确保您的反向传播是正确的，然后添加dropout。
   
   恭喜，您可以确保您的欺诈检测深度学习模型完全正确！你甚至可以用它来说服你的CEO。 :) 
   
   <font color ='blue'> 
    
 **你应该记住的笔记**： 
    - 梯度检验验证来自反向传播的梯度与梯度的数值近似（使用前向传播计算）之间的接近程度。 
    - 梯度检验很慢，因此我们不会在每次训练迭代中运行它。通常只需运行它以确保您的代码是正确的，然后将其关闭并使用backprop进行实际的学习过程。